# Домашнє завдання Тема 3

### 1. Визначте DataFrame з тривимірними векторами слів:

*   Завантажте модель word embeddings, використовуючи файл, який містить NLP модель.
*   Витягніть тривимірні вектори для всіх слів з цієї моделі.
*   Створіть DataFrame, в якому буде міститися інформація про слова та їхні тривимірні вектори.

In [51]:
import pandas as pd # Library for Dataframes
import numpy as np # Library for math functions
import pickle # Python object serialization library. Not secure
import requests
import io



# Завантажте модель word embeddings, використовуючи файл, який містить NLP модель.

url = 'https://drive.google.com/uc?export=download&id=1281E0CDneuKdflWFBUvuyUzujpdGVImz'

# Download the file
response = requests.get(url)

# Load the content into a BytesIO object
with io.BytesIO(response.content) as f:
    word_embeddings = pickle.load(f)

len(word_embeddings) # there should be 243 words that will be used in this assignment

243

In [52]:
from sklearn.decomposition import PCA

# Витягніть тривимірні вектори для всіх слів з цієї моделі.

embedding_matrix = np.array(list(word_embeddings.values()))

print(embedding_matrix[0:5])
print(embedding_matrix.shape)

pca = PCA(n_components=3)
reduced_embeddings = pca.fit_transform(embedding_matrix)

print(reduced_embeddings[0:5])
print(reduced_embeddings.shape)

[[-0.08007812  0.13378906  0.14355469 ... -0.04418945  0.09716797
   0.06738281]
 [-0.0100708   0.05737305  0.18359375 ... -0.12255859  0.07617188
  -0.234375  ]
 [-0.07324219  0.13574219  0.10888672 ...  0.13769531  0.09375
  -0.07958984]
 [ 0.19140625  0.125      -0.06542969 ... -0.0546875   0.30273438
   0.10595703]
 [-0.13964844  0.06225586 -0.27929688 ...  0.20410156  0.23535156
  -0.05102539]]
(243, 300)
[[-0.74603677 -0.3879644  -0.48269066]
 [-0.10249414  0.1403837  -1.1898899 ]
 [-0.83105475 -0.12950021 -0.31259927]
 [-0.65633696 -0.17779104 -0.3383812 ]
 [-0.6583452  -0.43246353 -0.6212126 ]]
(243, 3)


In [53]:
# Створіть DataFrame, в якому буде міститися інформація про слова та їхні тривимірні вектори.

reduced_word_embeddings = {word: reduced_embeddings[i] for i, word in enumerate(word_embeddings.keys())}
print(reduced_word_embeddings)

df = pd.DataFrame(reduced_word_embeddings.values(), index=reduced_word_embeddings.keys())
df.head()

{'country': array([-0.74603677, -0.3879644 , -0.48269066], dtype=float32), 'city': array([-0.10249414,  0.1403837 , -1.1898899 ], dtype=float32), 'China': array([-0.83105475, -0.12950021, -0.31259927], dtype=float32), 'Iraq': array([-0.65633696, -0.17779104, -0.3383812 ], dtype=float32), 'oil': array([-0.6583452 , -0.43246353, -0.6212126 ], dtype=float32), 'town': array([-0.37807617, -0.08044366, -1.0534686 ], dtype=float32), 'Canada': array([-1.1919613 , -0.40770915, -0.74181634], dtype=float32), 'London': array([ 0.29798645,  0.2533871 , -1.1518635 ], dtype=float32), 'England': array([-1.1039779 , -0.11007825, -0.89342946], dtype=float32), 'Australia': array([-1.049533  , -0.45226678, -0.8357717 ], dtype=float32), 'Japan': array([-1.0374848 , -0.09764805, -0.7604673 ], dtype=float32), 'Pakistan': array([-0.69454414, -0.48047915,  0.0989624 ], dtype=float32), 'Iran': array([-0.68421555,  0.14266318,  0.36559993], dtype=float32), 'gas': array([-0.7913052 ,  0.12190384, -0.591116  ], dt

,0,1,2
country,-0.746037,-0.387964,-0.482691
city,-0.102494,0.140384,-1.189890
China,-0.831055,-0.129500,-0.312599
Iraq,-0.656337,-0.177791,-0.338381
oil,-0.658345,-0.432464,-0.621213


In [54]:
# # Створіть DataFrame, в якому буде міститися інформація про слова та їхні тривимірні вектори.

# df = pd.DataFrame(reduced_embeddings, columns=['Dimension 1', 'Dimension 2', 'Dimension 3'])
# df['Word'] = list(word_embeddings.keys())

# # Переміщаємо стовпець з словами на перше місце
# df = df[['Word', 'Dimension 1', 'Dimension 2', 'Dimension 3']]
# df.head()

In [55]:
import matplotlib.pyplot as plt # Import matplotlib

df['Word'] = df.index  # Додаємо слова як окремий стовпець

import plotly.express as px
fig = px.scatter_3d(df, x=0, y=1, z=2,
              color='Word')
fig.show()

### 2. Визначте функції для пошуку найближчого слова:

*   Напишіть функцію, яка приймає тривимірний вектор та знаходить найближче слово в моделі word embeddings_subset.
*   Використайте цю функцію для кількох прикладів та переконайтеся, що результати є коректними.

In [56]:
from scipy.spatial.distance import cosine

# Функція приймає тривимірний вектор та знаходить слово або найближче слово в reduced_word_embeddings.
def find_word(target_vector):

    # Ініціалізація змінних для пошуку найближчого слова
    closest_word = None
    min_distance = float('inf')

    # Пошук найближчого слова
    for word, vector in reduced_word_embeddings.items():    # reduced_word_embeddings = dict 3D-vectors from global (Словник зі зменшеними векторами)
        # Обчислюємо косинусну відстань
        distance = cosine(target_vector, vector)
        if distance < min_distance:
            min_distance = distance
            closest_word = word

    return closest_word


In [57]:
# Напишіть функцію, яка приймає слово та знаходить найближче йому за семантикою (виключаючи точне попадання) в reduced_word_embeddings.
def find_semantic_closest_word(word):
  try:

    # Ініціалізація змінних для пошуку найближчого слова
    target_vector = reduced_word_embeddings[word]
    closest_word = None
    min_distance = float('inf')

    # Пошук найближчого слова
    for word, vector in reduced_word_embeddings.items():    # reduced_word_embeddings = dict 3D-vectors from global (Словник зі зменшеними векторами)
        # Пропускаємо вектор, якщо він співпадає з цільовим
        if np.array_equal(vector, target_vector):
            continue

        # Обчислюємо косинусну відстань
        distance = cosine(target_vector, vector)
        if distance < min_distance:
            min_distance = distance
            closest_word = word

    return closest_word
  except KeyError as e:
      (f"KeyError: The word {e} was not found in the embeddings subset.")
      return None
  except Exception as e:
      print(f"An error occurred: {e}")
      return None

In [58]:
# приклади використання.
print('Найближче слово до вектора [0.1, 0.2, 0.3]:', find_word(np.array([0.1, 0.2, 0.3])))
print('Найближче слово до вектора [0.5, 0.5, 0.5]:', find_word([0.5, 0.5, 0.5]))
print('Cлово, що відповідає о вектору [0.66962737 1.7194687  0.6131421 ]:', find_word(np.array([0.66962737, 1.7194687,  0.6131421 ])))
print('Знайти "Tokyo" через його вектор:', find_word(reduced_word_embeddings['Tokyo']))
print('Найближче слово до рандомного вектора:', find_word(np.random.rand(3)))

print()
print('Слово найближче за семантикою до "Tokyo":', find_semantic_closest_word('Tokyo'))
print('Слово найближче за семантикою до "Sudan":', find_semantic_closest_word('Sudan'))
print('Слово найближче за семантикою до "Slovo":', find_semantic_closest_word('Slovo'))

Найближче слово до вектора [0.1, 0.2, 0.3]: Chisinau
Найближче слово до вектора [0.5, 0.5, 0.5]: Ashgabat
Cлово, що відповідає о вектору [0.66962737 1.7194687  0.6131421 ]: Kiev
Знайти "Tokyo" через його вектор: Tokyo
Найближче слово до рандомного вектора: Rabat

Слово найближче за семантикою до "Tokyo": Paris
Слово найближче за семантикою до "Sudan": Zambia
Слово найближче за семантикою до "Slovo": None


### 3. Обчисліть векторний добуток для знаходження ортогонального слова:

*   Виберіть декілька довільних пар слів.
*   Обчисліть векторний добуток для кожної пари слів та використайте раніше написану функцію для знаходження найближчого слова.
*   Проаналізуйте результати та спробуйте їх інтерпретувати.

In [59]:
import random

for i in range(3):
  # Вибір пари слів.
  pair = random.sample(list(reduced_word_embeddings.keys()), 2)
  ort_vect = np.cross(reduced_word_embeddings[pair[0]], reduced_word_embeddings[pair[1]])
  # Знаходження найближчого слова.
  print(f'Для {pair} ортгональним є {find_word(ort_vect)}')

Для ['Oman', 'Canberra'] ортгональним є Gaborone
Для ['Madagascar', 'Suriname'] ортгональним є Roseau
Для ['Philippines', 'Belize'] ортгональним є Eritrea


## Аналіз результатів
*   Для пари ['Valletta', 'Cairo']: Ортогональним словом є Lisbon.
Це мало б означати, що вектор Лісабону в певному сенсі балансує характеристики Валетти та Каїру, як у розмірності, які залишаються після PCA.
*   Для пари ['Ireland', 'Kampala']: Ортогональним словом є Tajikistan.
Тут можемо спостерігати, що Таджикистан, хоча і географічно і культурно відрізняється від Ірландії та Кампали, має вектор у просторі, який певним чином "доповнює" цю пару.
*   Для пари ['England', 'Zimbabwe']: Ортогональним словом є Belarus.
В цьому випадку Білорусь обрана як вектор, що може мати певні загальні особливості з Англією та Зімбабве або займає унікальне положення, яке поєднує характеристики обох.

## Інтерпретація
*   **Семантична близькість:** Слова, що вважаються "ортогональними" в цьому контексті, не обов'язково мають очевидні семантичні зв'язки, а натомість відображають математичну особливість розташування векторів у зменшеному просторі.

*   **Обмеження векторного простору**: Можливо, після зменшення розмірності через PCA деякі нюанси семантичного змісту можуть бути втрачені, і результати можуть відображати більше математичні властивості.


## Висновок
Цей аналіз показує, як різні слова в NLP моделі можуть мати цікаві взаємозв'язки через їх векторні представлення. Однак інтерпретація цих зв'язків повинна враховувати як семантичний, так і математичний контексти. Це підкреслює, як складно повністю передати людське розуміння через математичні моделі.

### 4. Напишіть функції визначення кута між словами:

*   Розробіть функцію, яка обчислює кут між векторами для довільних двох слів.
*   Протестируйте цю функцію для різних пар слів.
*   Розгляньте отримані результати та спробуйте визначити їхню інтерпретацію.

In [60]:
def angle(vector1, vector2):
    dot_product = np.dot(vector1, vector2)
    norm1 = np.linalg.norm(vector1)
    norm2 = np.linalg.norm(vector2)

    # Обчислення косинуса кута
    cos_theta = dot_product / (norm1 * norm2)

    # Обчислення кута у радіанах
    theta_radians = np.arccos(np.clip(cos_theta, -1.0, 1.0))

    # Перетворення кута у градуси
    theta_degrees = np.degrees(theta_radians)

    return theta_degrees

import random
for i in range(3):
    # Вибір пари слів.
    pair = random.sample(list(reduced_word_embeddings.keys()), 2)
    vector1 = reduced_word_embeddings[pair[0]]
    vector2 = reduced_word_embeddings[pair[1]]
    # print(angle(vector1, vector2))
    print(f'Між {pair} кут {angle(vector1, vector2)}')

print(f'Між "Paris", "France" кут {angle(reduced_word_embeddings["Paris"], reduced_word_embeddings["France"])}')
print(f'Між "Kiev", "Minsk" кут {angle(reduced_word_embeddings["Minsk"], reduced_word_embeddings["Kiev"])}')
print(f'Між "Ukraine", "Armenia" кут {angle(reduced_word_embeddings["Ukraine"], reduced_word_embeddings["Armenia"])}')

    # ort_vect = np.cross(reduced_word_embeddings[pair[0]], reduced_word_embeddings[pair[1]])
    # Знаходження найближчого слова.


Між ['Khartoum', 'Armenia'] кут 112.89978660054813
Між ['Belmopan', 'Maputo'] кут 45.823639606848985
Між ['Guinea', 'Morocco'] кут 37.44804037392084
Між "Paris", "France" кут 99.61886575818527
Між "Kiev", "Minsk" кут 7.952112792621063
Між "Ukraine", "Armenia" кут 10.041027879407153


## Результати

*   Кут між ['Algiers', 'Nairobi'] — 16.34 градусів. Цей кут досить малий, що свідчить про те, що Algiers (столиця Алжиру) і Nairobi (столиця Кенії) мають певну семантичну схожість. Це може відображати їхню роль як столиць країн Африки, які можуть мати подібні культурні, економічні або географічні асоціації в текстових даних, на яких була навчена модель.
*   Кут між ['Doha', 'Dhaka'] — 48.02 градусів. Кут помірний, що вказує на деяку схожість, але також і на відмінності. Doha (столиця Катару) та Dhaka (столиця Бангладеш) обидва є столицями, але розташовані в різних географічних регіонах з різними культурними та економічними контекстами. Схожість може бути обумовлена загальними характеристиками, притаманними столицям.
*   Кут між ['Beijing', 'Namibia'] — 162.14 градусів. Дуже великий кут вказує на те, що Beijing (столиця Китаю) і Namibia (країна в Південно-Західній Африці) мають значну семантичну різницю в обраному просторі embeddings. Це може бути через відмінності в їх географічному розташуванні, економічних ролях, культурних або політичних характеристиках, що відображено в текстах, на яких навчалася модель.
*   Кут між ['Paris', 'France'] — 99.62 градусів. Paris (столиця Франції) і France мають кут трохи менше 100 градусів, що вказує на суттєву різницю. Це може здатися неочікуваним, але такі результати можуть виникати між країною і столицею.
*   Кут між ['Kiev', 'Minsk'] — 7.95 градусів. Kiev (столиця України) та Minsk (столиця Білорусі) мають дуже малий кут, що вказує на високу семантичну схожість. Це може бути обумовлено близькістю країн, схожими історичними та культурними контекстами.
*   Кут між ['Ukraine', 'Armenia'] — 10.04 градусів. Дуже малий кут, по тій же причині.

## Висновок
Як бачимо, ми отримуємо малий кут між судніми країнами чи сусідніми країнами внаслідок географічної близькості, схожості історії та культури, проте між країноє і столицею (навіть цієї ж країни), кут великий. **Виходить, що різниця в типах адміністративних одиниць більш помітна моделі тіж териториальна.**

Оскільки, модель навчалася на основі згадування в тексті, де, зазвичай, країни перелічуються з країнами, а столиці зі столицями і навряд чи змішуються.
А може, "розуміння" зв'язку столиціз країною втратилося при зменшенні розмірності   

Отримані результати демонструють, як векторні представлення можуть виявляти семантичні зв'язки та відмінності між словами, ґрунтуючись на контексті, в якому вони використовуються в текстах, на яких навчалася модель word embeddings.